In [9]:
%reset

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
#this cell is only for using with google colab
import sys
from google.colab import drive
!git clone https://github.com/kohmab/nn-steady-state-discharge.git
drive.mount('/content/gdrive')
sys.path.append('/content/nn-steady-state-discharge/')

Cloning into 'nn-steady-state-discharge'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 162 (delta 68), reused 134 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (162/162), 2.10 MiB | 4.21 MiB/s, done.
Resolving deltas: 100% (68/68), done.


ModuleNotFoundError: No module named 'google.colab'

In [12]:
import torch
import numpy as np
from configuration import PARAMETERS

torch.manual_seed(PARAMETERS.torch.seed)


In [13]:
from networks import Pidonet

model = Pidonet().to(device=PARAMETERS.torch.device, dtype=PARAMETERS.torch.dtype)

s = 0
for p in model.parameters():
    s += p.numel()
print(f"Parameters count is {s}.")

model_path = "beam_september_pidonet.pth"

Parameters count is 59010.


In [14]:
bs = PARAMETERS.data.batch_size.train

from data import *

optimizerA = torch.optim.SGD(model.parameters(), lr=1.e-4)
dataloaders = JointDataLoaders(
    bs,
    "ic",
    {"ic": IcData(bs),
     "bc_axis": BcAtAxisData(bs),
     "bc_top": BcAtUpperRLimitData(bs)}
)

In [15]:
from training import EqLossesWithWeights

N_POINTS_ON_SEGMENT = PARAMETERS.data.num_of_points.eq // PARAMETERS.training.z_segments

eq_losses_with_weights = EqLossesWithWeights()
l_eq_at_segments = torch.empty(PARAMETERS.training.z_segments,
                               device=PARAMETERS.torch.device,
                               dtype=PARAMETERS.torch.dtype)
seg_lb = HYPERDOMAIN.lb.repeat(PARAMETERS.training.z_segments, 1)
seg_ub = HYPERDOMAIN.ub.repeat(PARAMETERS.training.z_segments, 1)
seg_z = torch.linspace(HYPERDOMAIN.zmin, HYPERDOMAIN.zmax, PARAMETERS.training.z_segments + 1)
seg_lb[:, 0] = seg_z[:-1]
seg_ub[:, 0] = seg_z[1:]
eq_data = SegmentData(N_POINTS_ON_SEGMENT)




In [16]:
def weight_loses(*losses):
    loss = torch.stack(losses)
    minimum = loss.min()
    weights = (1 + (loss - minimum) / minimum).pow(3).detach()
    loss = loss * weights
    return loss.sum()


In [20]:
from losses import *
from tqdm.notebook import tqdm
from IPython.display import clear_output

try:
    model = torch.load(model_path, weights_only=False)
    print("Model loaded.")
except FileNotFoundError:
    print("Model was not found.")

# TODO try to use training algorythm from [https://arxiv.org/abs/2308.08468]


for e in range(100500):
    for _ in dataloaders:
        optimizerA.zero_grad()
        seg_eq_losses = torch.empty(PARAMETERS.training.z_segments, device=PARAMETERS.torch.device,
                                    dtype=PARAMETERS.torch.dtype)
        for i in tqdm(range(PARAMETERS.training.z_segments)):
            seg_eq_losses[i] = eq_losses(model, *eq_data(seg_lb[i], seg_ub[i]))
        l_eq = eq_losses_with_weights(seg_eq_losses)
        l_bc_u = upper_r_bc_losses(model, *dataloaders.data["bc_top"])
        l_bc_a = axis_bc_losses(model, *dataloaders.data["bc_axis"])
        l_ic = ic_losses(model, *dataloaders.data["ic"])*100

        losses = l_eq + l_bc_u + l_bc_a + l_ic
        # w_losses = weight_loses(l_eq, l_bc_u, l_bc_a, l_ic)

        losses.backward()
        optimizerA.step()

    clear_output(False)
    tqdm.write(f"\n Epoch {e}")
    tqdm.write(f"train_losses = {losses}\n")
    tqdm.write(f"eq: {l_eq}, bc: {l_bc_a},{l_bc_u}, ic: {l_ic}\n")

    # TODO must check test losses
    torch.save(model, model_path)

    if losses < 1e-8:
        break




 Epoch 61
train_losses = 3.7449731826782227

eq: 0.00018981161701958627, bc: 7.650387851754203e-05,0.03666020557284355, ic: 3.7080466747283936



  0%|          | 0/128 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [21]:
from networks import AnalyticNet
from hyperdomain import HYPERDOMAIN
from representation import GridModelExecutor

zmin = HYPERDOMAIN.zmin
zmax = HYPERDOMAIN.zmax
rmin = HYPERDOMAIN.rmin
rmax = HYPERDOMAIN.rmax

max_field = 1
p = 1.
K0 = 1.
nu = 0.

z, r = np.meshgrid(np.linspace(zmin, zmax, 256), np.linspace(rmin, rmax, 256))

pinn = GridModelExecutor(model, max_field, p, K0, nu)
analyt = GridModelExecutor(AnalyticNet(), max_field, p, K0, nu)

pinn(z, r)
analyt(z, r)

Er_net = pinn.field.real
Ei_net = pinn.field.imag

Er_ex = analyt.field.real
Ei_ex = analyt.field.imag

n = pinn.plasma_density

In [22]:
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.use("notebook")
# plt.close('all')

fig = plt.figure(1, figsize=(8, 8))
plt.clf()
ax = fig.add_subplot(221)
cf = plt.contourf(z, r, Er_net)
fig.colorbar(cf)
ax.title.set_text(r"model Im$E$")

ax = fig.add_subplot(222)
cf = plt.contourf(z, r, Ei_net)
fig.colorbar(cf)
ax.title.set_text(r"model Re$E$")

ax = fig.add_subplot(223)
cf = plt.contourf(z, r, Er_ex)
fig.colorbar(cf)
ax.title.set_text(r"exact Re$E$ (without plasma)")

ax = fig.add_subplot(224)
cf = plt.contourf(z, r, Ei_ex)
fig.colorbar(cf)
ax.title.set_text(r"exact Im$E$ (without plasma)")

fig = plt.figure(2, figsize=(8, 8))
ax = fig.add_subplot(111)
cf = plt.contourf(z, r, n)
fig.colorbar(cf)
ax.title.set_text(r"Plasma density $n$")

plt.show()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>